# Simple Visualization of the Marked Input Point Cloud

This script was prepared to showcase the 3D point cloud at the DTU StartUp-Day in case interested people would like to interact with something.

In [2]:
# For paths
import os 

# For data processing
import torch
import numpy as np
from torch.utils.data import Dataset
import math


# For 3D visualization
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

# Setting the root for this repo
ROOT = os.path.realpath("..")

# Creating/ Making the point cloud
def makePC(point_data, color_data=np.array([])):
    pcd = o3d.geometry.PointCloud() #Create PC object
    pcd.points = o3d.utility.Vector3dVector(point_data) #Give coordinates
    #Coloring the PC
    if len(color_data) == 0:
        pcd.paint_uniform_color([1, 0, 0])
    else:
        pcd.colors = o3d.utility.Vector3dVector(color_data)
    return pcd

# Axis alignment of given point cloud
def axisAlignment(main_pcd):
    """Rotating/axis aligning the pointcloud
    input: main_pcd = initial pointcloud (open3d.geometry.PointCloud object)
    output: main_pcd = rotated pointcloud (open3d.geometry.PointCloud object)"""
    bbox = main_pcd.get_oriented_bounding_box()
    inverse_R = np.linalg.inv(bbox.R)
    main_pcd.rotate(inverse_R)
    return main_pcd

# Path creation for easier loading
root_based_input_pc_path = ROOT + "/data/testdata/data_labelled_int.npy"
input_pc_path = '/Users/nk/Documents/GitHubRepos/Pointnet_Pointnet2_pytorch/data/testdata/data_labelled_int.npy'

# Loading relevant data
input_pc = np.load(input_pc_path) #ndarray
input_pc_points, input_pc_labels = input_pc[:, 0:6], input_pc[:, 6]

# Assigning the coordinates
input_points = np.stack([input_pc_points[:, 0], 
                         input_pc_points[:, 1], 
                         input_pc_points[:, 2]], axis=1)
# Assigning color channels
red_c   = np.array(input_pc_points[:, 3])
green_c = np.array(input_pc_points[:, 4])
blue_c  = np.array(input_pc_points[:, 5])
# Converting color channels to [0, 1] range
red_c   = (red_c - np.min(red_c)) / (np.max(red_c) - np.min(red_c))
green_c = (green_c - np.min(green_c)) / (np.max(green_c) - np.min(green_c))
blue_c  = (blue_c - np.min(blue_c)) / (np.max(blue_c) - np.min(blue_c))
point_sizes = np.zeros(len(input_pc_points))
# Marking fracture points
for i in range(len(input_points)):
    if input_pc_labels[i] == 1:
        red_c[i] = 1
        green_c[i] = 0
        blue_c[i] = 0
        point_sizes[i] = 5
# Creating color data
color_data = np.stack([red_c, green_c, blue_c], axis=1)

# Make the point cloud to be displayed
viz_pcd = makePC(input_points, color_data)

# Cropping the point cloud using bounding boxes
bbox = viz_pcd.get_oriented_bounding_box()
bbox.color = (0, 1, 0) #bbox in green
viz_pcd_cropped = viz_pcd.crop(bbox)

o3d.visualization.draw([viz_pcd_cropped], point_size=3)

FEngine (64 bits) created at 0x7fbf2ef40000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display


The only thing that might be itneresting to look into at some point is how one can change the thickness/ size of the fracture points. During the quick assembly of this I could find out that the size is only controllable for the entire pc, when using the draw function. I believe there is a trick possibly in the creation of the point cloud, that possibly two have to be created with differing sizes or possibly there is a better way of overlaying two different point clouds with open3d...